In [ ]:
# Obtención de los datos
import pandas as pd

if __name__ == '__main__':
    hotel_df = pd.read_csv('files/hotel_bookings.csv')
    x, y = hotel_df.shape
    print(x, y)

In [ ]:
hotel_df.head()

In [ ]:
hotel_df.info()

In [ ]:
hotel_df.describe()

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
hotel_df.hist(bins=50, figsize=(20,15))
plt.show()


In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
hotel_df.boxplot(figsize=(38, 10))
plt.show()

In [ ]:
hotel_df['is_canceled'].hist(bins=2)
print(hotel_df['is_canceled'].value_counts())
print("proporcion de clases: %s %%" % (round(class_1 / class_2, 2)))

In [ ]:
# Creating a data set for train
# 90/10
import numpy as np
from zlib import crc32

def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]


if __name__ == '__main__':
    np.random.seed(1024)
    train_set, test_set = split_train_test(hotel_df, 0.2)
    train_set.to_csv('files/train.csv', index=False)
    test_set.to_csv('files/test.csv', index=False)
    print(len(train_set), len(test_set))

In [ ]:
#EDA
#Experimenting with atributes combiaitions
#New atributes
train_set = pd.read_csv('files/train.csv')
train_set['total_guests'] = train_set['adults'] + train_set['children'] + train_set['babies']
train_set['total_days'] = train_set['stays_in_week_nights'] + train_set['stays_in_weekend_nights']
train_set.shape

In [ ]:
country_data = pd.DataFrame(train_set.loc[train_set["is_canceled"] == 0]["country"].value_counts())
#country_data.index.name = "country"
country_data.rename(columns={"country": "Number of Guests"}, inplace=True)
total_guests = country_data["Number of Guests"].sum()
country_data["Guests in %"] = round(country_data["Number of Guests"] / total_guests * 100, 2)
country_data["country"] = country_data.index
country_data.loc[country_data["Guests in %"] < 4, "country"] = "Other"

# pie plot
fig = px.pie(country_data,
             values="Number of Guests",
             names="country",
             title="De que paises vienen la mayoria de los huespedes que si cancelaron",
             template="seaborn")
fig.update_traces(textposition="inside", textinfo="value+percent+label")
fig.show()

In [ ]:
import plotly.express as px
train_set = pd.read_csv('files/train.csv')

train_set = train_set[train_set['country'] != 'PRT']

country_data_cancelados = train_set.groupby(['country'])['is_canceled'].sum().reset_index().sort_values(by='is_canceled',ascending=False)
country_data_cancelados['% de cancelados'] = round(country_data_cancelados['is_canceled'] / max(country_data_cancelados['is_canceled']) * 100, 2)


guest_map = px.choropleth(country_data_cancelados,
                    locations=country_data_cancelados['country'],
                    color=country_data_cancelados["% de cancelados"], 
                    hover_name=country_data_cancelados['country'], 
                    color_continuous_scale=px.colors.sequential.Reds,
                    title="Países donde existen mas cancelaciones (se excluye a Portugal)")
guest_map.show()

In [ ]:
#Por mes
meses_ordenados = ['January', "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
train_set['Mes'] = pd.Categorical(train_set['arrival_date_month'], categories=meses_ordenados, ordered=True)

country_data_cancelados = train_set.groupby(['country','Mes'])['total_guests','is_canceled'].sum().reset_index().sort_values(by='Mes')
country_data_cancelados['% de cancelados'] = round(country_data_cancelados['is_canceled']/ country_data_cancelados['total_guests'] * 100, 2)
country_data_cancelados.dropna(inplace=True)
country_data_cancelados = country_data_cancelados.reset_index()
country_data_cancelados = country_data_cancelados.drop('index',axis=1)

guest_map = px.choropleth(country_data_cancelados,
                    locations=country_data_cancelados['country'],
                    color=country_data_cancelados["total_guests"], 
                    hover_name=country_data_cancelados['country'],
                    animation_frame="Mes",
                    color_continuous_scale=px.colors.sequential.Reds,
                    title="Origen por fecha de paises donde existen mas cancelaciones")
guest_map.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 2000

guest_map.show()

In [ ]:
# Counting adults and children as paying guests only, not babies.
import numpy as np
train_set = pd.read_csv('files/train.csv')
rhotel = train_set[train_set['hotel'] == 'Resort Hotel'].copy()
chotel = train_set[train_set['hotel'] == 'City Hotel'].copy()

rhotel.fillna(1, inplace=True)
rhotel.replace(np.inf, 1, inplace=True)
rhotel["adr_pp"] = rhotel["adr"] / (rhotel["adults"] + rhotel["children"])
chotel.fillna(0, inplace=True)
chotel.replace(np.inf, 0, inplace=True)
chotel["adr_pp"] = chotel["adr"] / (chotel["adults"] + chotel["children"])

room_prices_mothly = train_set[["hotel", "arrival_date_month", "adr_pp"]].sort_values("arrival_date_month")

# order by month:
ordered_months = ["January", "February", "March", "April", "May", "June", 
          "July", "August", "September", "October", "November", "December"]
room_prices_mothly["arrival_date_month"] = pd.Categorical(room_prices_mothly["arrival_date_month"], categories=ordered_months, ordered=True)

# barplot with standard deviation:
plt.figure(figsize=(12, 8))
sns.lineplot(x = "arrival_date_month", y="adr_pp", hue="hotel", data=room_prices_mothly, 
            hue_order = ["City Hotel", "Resort Hotel"], ci="sd", size="hotel", sizes=(2.5, 2.5))
plt.title("Room price per night and person over the year", fontsize=16)
plt.xlabel("Month", fontsize=16)
plt.xticks(rotation=45)
plt.ylabel("Price [EUR]", fontsize=16)
plt.show()

In [ ]:
corr_matrix = train_set.corr()
corr_matrix['is_canceled'].sort_values(ascending=False)

In [ ]:
atributes = ['lead_time', 'previous_cancellations', 'total_of_special_requests', 'required_car_parking_spaces', 'booking_changes']
pd.plotting.scatter_matrix(train_set[atributes], figsize=(12, 8))

In [ ]:
train_set.isnull().sum() / len(train_set) * 100

In [ ]:
#Preparación de los datos
#X_train = pd.read_csv('files/train.csv')
X_train = train_set.drop('is_canceled', axis=1)
y_train = train_set['is_canceled'].copy()
print(X_train.shape, y_train.shape)

In [ ]:
X_train_num = X_train.select_dtypes(exclude=['object'])
print(X_train_num.columns)
#X_train_num[''].unique()
X_train_num.describe()

In [ ]:
X_train_cat = X_train.select_dtypes(include=['object', 'category'])
atrs_cat = X_train_cat.columns
atrs_cat

In [ ]:
X_train_cat.describe()

In [ ]:
#Missing values
from sklearn.impute import SimpleImputer
#For standarization
from sklearn.preprocessing import StandardScaler
#For work encode categorical atrubuts
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
#For do a best a work flow
from sklearn.pipeline import Pipeline
#Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
import pandas as pd

if __name__ == '__main__': 
    ########################################################################
    #Load our data sets
    train_set = pd.read_csv('files/train.csv')
    test_set = pd.read_csv('files/test.csv')
    
    #New atributes
    
    #Train
    train_set['total_guests'] = train_set['adults'] + train_set['children'] + train_set['babies']
    train_set['total_days'] = train_set['stays_in_week_nights'] + train_set['stays_in_weekend_nights']
    #Test
    test_set['total_guests'] = test_set['adults'] + test_set['children'] + test_set['babies']
    test_set['total_days'] = test_set['stays_in_week_nights'] + test_set['stays_in_weekend_nights']
    
    #Preparing data for our model
    
    #Train
    X_train = train_set.drop('is_canceled', axis=1)
    y_train = train_set['is_canceled'].copy()
    #Train
    X_test = test_set.drop('is_canceled', axis=1)
    y_test = test_set['is_canceled'].copy()
    
    #Cleaning data
    
    #Numerical atributs droped
    atrs_n = ['arrival_date_year', 'arrival_date_day_of_month', 'stays_in_weekend_nights', 'stays_in_week_nights', 'adults', 'children', 'babies', 'company', 'agent', 'total_guests']
    #Categorical atributs droped 
    atrs_cat = ['reservation_status', 'reservation_status_date', 'country', 'market_segment', 'distribution_channel', 'reserved_room_type', 'assigned_room_type']
    atrs = atrs_cat + atrs_n
    #Train
    X_train = X_train.drop(atrs, axis=1)
    #Test
    X_test = X_test.drop(atrs, axis=1)
    
    #Encoding category type data
    
    #########
    #Train
    X_train_num = X_train.select_dtypes(exclude=['object', 'category']).columns
    X_train_cat = X_train.select_dtypes(include=['object', 'category']).columns
    num_pipeline = Pipeline([
    #('imputer', SimpleImputer(strategy="constant")),
    #('attribs_adder', CombinedAttributesAdder()), #Experimenting with atributes combinations
    ('std', StandardScaler()),#std_scaler#Standarization
    ])

    num_attribs = X_train_num#For get numeric data
    cat_attribs = X_train_cat#For category data
    full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
    ]) 
    X_train = full_pipeline.fit_transform(X_train)
    #########
    #Test
    X_test_num = X_test.select_dtypes(exclude=['object', 'category']).columns
    X_test_cat = X_test.select_dtypes(include=['object', 'category']).columns
    num_pipeline = Pipeline([
    #('imputer', SimpleImputer(strategy="constant")),
    #('attribs_adder', CombinedAttributesAdder()), #Experimenting with atributes combinations
    ('std', StandardScaler()),#std_scaler#Standarization
    ])

    num_attribs = X_test_num#For get numeric data
    cat_attribs = X_test_cat#For category data
    full_pipeline = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", OneHotEncoder(), cat_attribs),
    ]) 
    
    X_test = full_pipeline.fit_transform(X_test)
    print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
    
    ########################################################################   
    ####### Logit Regression ######
    log_reg = LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                    intercept_scaling=1, l1_ratio=None, max_iter=500,
                    multi_class='multinomial', n_jobs=-1, penalty='l2',
                    random_state=42, solver='lbfgs', tol=0.000001, verbose=0,
                    warm_start=False)
    log_reg.fit(X_train, y_train)
    #y_predict = log_reg.predict(X_test)
    #######acc = 0.7884244911634141
   
    #####################################################
    #Decission TeeeClassifer
    tree_clas = DecisionTreeClassifier(ccp_alpha=0.0000001, criterion='gini',
                                       max_depth=33, max_features=9, max_leaf_nodes=None,
                                       min_impurity_decrease=0.0, min_impurity_split=None,
                                       min_samples_leaf=1, min_samples_split=2,
                                       min_weight_fraction_leaf=0.0, presort='deprecated',
                                       random_state=42, splitter='best')
    tree_clas.fit(X_train, y_train)
    #y_predict = tree_clas.predict(X_test)   
    ######.86
    #####################################################
    ####### Decission RandomForestClassifier ######
    forest_reg = RandomForestClassifier(n_estimators=292,
                                        criterion='gini',
                                        max_depth=32,
                                        min_samples_split=2,
                                        min_samples_leaf=1, 
                                        min_weight_fraction_leaf=0.0, 
                                        max_features=8, 
                                        max_leaf_nodes=None, 
                                        min_impurity_decrease=0.0, 
                                        min_impurity_split=None, 
                                        bootstrap=True, 
                                        oob_score=True, 
                                        n_jobs=-1, 
                                        random_state=42, 
                                        verbose=0, 
                                        warm_start=False, 
                                        class_weight=None, 
                                        ccp_alpha=0.000001, 
                                        max_samples=None, 
                                       )
    forest_reg.fit(X_train, y_train)
    #y_predict = rand_clas.predict(X_test)    
    ######0.8546035024693207######
    #####################################################
    ###### Decission SVM ######
    svm_clf = svm.SVC(C=1.0, break_ties=False, cache_size=800, class_weight=None, coef0=0.0,
                        decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
                        max_iter=-1, probability=True, random_state=42, shrinking=True, tol=0.001,
                        verbose=False)
    svm_clf.fit(X_train, y_train)
    #y_predict = svm_clas.predict(X_test)    
    #######acc = 0.8045736304238565######



(95512, 37) (95512,) (23878, 37) (23878,)


In [ ]:
print(accuracy_score(y_predict, y_test))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, KFold, cross_validate, cross_val_score

# define models to test:
base_models = [("DT_model", DecisionTreeClassifier(random_state=42)),
               ("RF_model", RandomForestClassifier(random_state=42,n_jobs=-1)),
               ("LR_model", LogisticRegression(random_state=42,n_jobs=-1)),
               #("XGB_model", XGBClassifier(random_state=42, n_jobs=-1))
              ]

param_grid = {'n_estimators': [200, 500],
              'max_features': ['auto', 'sqrt', 'log2'],
              'max_depth' : [4,5,7],
              'criterion' :['gini', 'entropy']
             }
forest_reg = RandomForestClassifier(random_state=1024, n_jobs=-1)
grid_search = GridSearchCV(estimator=forest_reg, param_grid=param_grid, cv=5, verbose=True)
grid_search.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import classification_report
final_model = grid_search.best_estimator_
y_predict = final_model.predict(X_test)
print(classification_report(y_test, y_predict))

# AUC ROC

In [ ]:
#AUC
from sklearn.metrics import roc_auc_score
AUC_log_reg = roc_auc_score(y_test, y_predict_lr)
AUC_tree_clas = roc_auc_score(y_test, y_predict_dt)
AUC_rand_clas = roc_auc_score(y_test, y_predict_rf)
AUC_svm_clas = roc_auc_score(y_test, y_predict_svm)

In [ ]:
from sklearn.metrics import roc_curve
#Logistic Regresion
y_pred_lr = log_reg.predict_proba(X_test_lr)[:, 1]
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_pred_lr)

#Decission TeeeClassifer
y_pred_dt = tree_clas.predict_proba(X_test_dt)[:,1]
fpr_tree_clas, tpr_tree_clas, _ = roc_curve(y_test, y_pred_dt)

#Decission RandomForestClassifier
y_pred_rf = forest_reg.predict_proba(X_test_rf)[:,1]
fpr_rand_clas, tpr_rand_clas, _ = roc_curve(y_test, y_pred_rf)

#Decission SVM
y_pred_svm_clas = svm_clf.predict_proba(X_test_svm)[:,1]
fpr_svm_clas, tpr_svm_clas, _ = roc_curve(y_test, y_predict_svm)

In [ ]:
from matplotlib import pyplot as plt
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')

plt.plot(fpr_lr, tpr_lr, label='Logistic Regression')
plt.plot(fpr_tree_clas, tpr_tree_clas, label='Decission Tree')
plt.plot(fpr_rand_clas, tpr_rand_clas, label='Random Forest')
plt.plot(fpr_svm_clas, tpr_svm_clas, label='SVM')

plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.grid(True)
plt.show()